In [ ]:
import copy


In [ ]:
if __name__ == '__main__':   
    %run Tests.ipynb
    %run MoleculeGenerator2.ipynb
    %run Discrim.ipynb
    %run Rewards.ipynb
    %run PPO_WITH_TRICKS.ipynb
    %run ChemEnv.ipynb
    %run SupervisedPreTraining.ipynb

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
# wants: a single class for pretraining and rl training
#     also want a singler logger for everything
#     also should put in cross validation for the supervised portion
#     means a logger instance in the init method

class SupervisedToReinforcement():
    def __init__(self,run_title, rewards_list, chem_env_kwargs, PPO_kwargs, svw_kwargs):
        
        self.run_title = run_title
        self.writer = SummaryWriter(f'./tb_logs/{run_title}/{run_title}_logs')
        
        self.reward_module =  FinalRewardModule(self.writer,rewards_list)
        
        chem_env_kwargs['num_chunks'] = train_kwargs['num_chunks']
        chem_env_kwargs['RewardModule'] = self.reward_module
        chem_env_kwargs['writer'] = self.writer
        
        self.ChemEnv = ChemEnv(**chem_env_kwargs)
        
        
        input_dim = chem_env_kwargs['num_node_feats']
        
        #self.policy = Spin2(input_dim,300,chem_env_kwargs['num_atom_types']).cuda()
        self.policy = BaseLine(input_dim,800,chem_env_kwargs['num_atom_types']+1).cuda()
        self.policy.apply(init_weights_recursive)
        
        
        
        svw_kwargs['writer'] = self.writer
        svw_kwargs['input_dim'] = input_dim
        svw_kwargs['num_atom_types'] = chem_env_kwargs['num_atom_types']
        
        print(svw_kwargs)
        self.svw = Supervised_Trainer(self.policy, **svw_kwargs)
        
        PPO_kwargs['env'] = self.ChemEnv
        PPO_kwargs['actor'] = self.policy
        PPO_kwargs['writer'] = self.writer
        self.PPO = PPO_MAIN(**PPO_kwargs)
        self.PPO.to_device(device)
        
    
    def Train(self,total_epochs, batch_size, epochs_per_chunk, num_chunks, PPO_steps, cv_path):
        
        self.svw.TrainModel(total_epochs)
    
#         torch.save({
#             'model_state_dict': self.policy.state_dict(),
#             'optimizer_state_dict': self.svw.optim.state_dict()
#             }, f'./{self.run_title}/SavedModel')
        
        print("fra")
#         self.PPO.learn(PPO_steps)
        

In [ ]:
%run SupervisedPreTraining.ipynb

In [ ]:
# rewards_list = [SizeSynth_norm()]
# rewards_list = [Synthesizability(), SizeReward()]
rewards_list = [ Synthesizability()]

chem_env_kwargs = {'max_nodes' : 12, 
                   'num_atom_types' : 17, 
                   'num_node_feats' : 54,
                   'num_edge_types' : 3, 
                   'bond_padding' : 12, 
                   'mol_featurizer': mol_to_graph_full, 
                   'RewardModule' : None, 
                   'writer' : None}


PPO_kwargs = {'env' : None,
              'batch_size' : 32,
              'timesteps_per_batch' : 1200,
              'clip' : 0.08,
              'a_lr' : 1e-4,
              'c_lr' : 3e-4,
              'n_updates_per_iteration' : 6,
              'max_timesteps_per_episode' : 40,
              'gamma' : .95,
              'actor' : None}
svw_kwargs = {'batch_size' : 128, 'data_set_size' : 507528}
train_kwargs = {'total_epochs' : 15,
                'batch_size' : 256,
                'epochs_per_chunk' : 1,
                'num_chunks' : 0,
                'cv_path' : './CrossVal/chunk_11',
                'PPO_steps' : 150000}


%run ChemEnv.ipynb
svtr = SupervisedToReinforcement('test_18',rewards_list,chem_env_kwargs,PPO_kwargs,svw_kwargs)
svtr.Train(**train_kwargs)



In [ ]:
Chem.MolFromSmiles('CCCN(CC)C(=O)S')

In [ ]:
svtr.PPO.inference()

In [ ]:
env = svtr.ChemEnv

In [ ]:
env.assignMol(Chem.MolFromSmiles('CCC(C)C(=O)O'))
print(env.last_action_node)
env.StateSpace

In [ ]:
env.step(0,verbose=True)
env.StateSpace

In [ ]:
PPO_kwargs = {'env' : env,
              'batch_size' : 32,
              'timesteps_per_batch' : 1200,
              'clip' : 0.08,
              'a_lr' : 1e-4,
              'c_lr' : 3e-4,
              'n_updates_per_iteration' : 6,
              'max_timesteps_per_episode' : 40,
              'gamma' : .95,
              'actor' : svtr.svw.policy,
             'writer': SummaryWriter(f'./tb_logs/3/3_logs')}
ppo_test = PPO_MAIN(**PPO_kwargs)
ppo_test.inference()


In [ ]:
Chem.MolFromSmiles('CCC(C(N)=O)N1CC(C)CC1=O')

In [ ]:
Chem.MolFromSmiles('CCCNC(=O)n1ccnc1C')

In [ ]:
env.assignMol(Chem.MolFromSmiles('C.C'))
env.step(19,verbose=True)
env.StateSpace

In [ ]:
chem_env_kwargs = {'max_nodes' : 12, 
                   'num_atom_types' : 17, 
                   'num_node_feats' : 54,
                   'num_edge_types' : 3, 
                   'bond_padding' : 12, 
                   'mol_featurizer': mol_to_graph_full, 
                   'RewardModule' : rewards_list, 
                   'writer' : SummaryWriter(f'./tb_logs/3/3_logs'),
                   'num_chunks': 1}

In [ ]:
%run ChemEnv.ipynb

In [ ]:
env = ChemEnv(**chem_env_kwargs)

In [ ]:
env.assignMol(Chem.MolFromSmiles('CCC.N'))

In [ ]:
env.step(2, verbose=True)
env.StateSpace

In [ ]:
ppo_test = PPO_MAIN(**PPO_kwargs)

In [ ]:
svtr.PPO.actor = svtr.policy

In [ ]:
Chem.MolFromSmiles('CCC.N')

In [ ]:
ppo_test.inference(True)

In [ ]:
torch.save({
            'model_state_dict': svtr.policy.state_dict(),
            'optimizer_state_dict': svtr.svw.optim.state_dict()
            }, './test_1/ah')

In [ ]:
svtr.policy.state_dict()
model = Spin2(54,300,17)
model.load_state_dict(svtr.policy.state_dict())

In [ ]:
%run ChemEnv.ipynb
svtr = SupervisedToReinforcement('test',rewards_list,chem_env_kwargs,PPO_kwargs)
env = svtr.ChemEnv

In [ ]:
svtr.PPO.inference()

In [ ]:
torch.save(svtr.PPO.actor.state_dict(), './model')

In [ ]:
env = svtr.ChemEnv
env.reset()
env.step(14)
env.step(17)
env.step(14)
env.StateSpace

In [ ]:
(Chem.MolFromSmiles('NCc1cccc([SH]=O)c1', sanitize = True))


In [ ]:
Chem.MolFromSmiles('Nc1cc2ccc1SSC(S)C2O.c1ccnc1', sanitize = False)

In [ ]:


env.reset()
#env.StateSpace = Chem.RWMol(Chem.MolFromSmiles('Nc1cc2ccc1SSC(S)C2O.c1ccnc1', sanitize = False))
#env.step(16)

#env.addEdge(1,0)

env.addBenzine()
env.addEdge(1,0)
env.StateSpace
env.addPyrrole()
env.addEdge(1,11) 
# env.StateSpace.RemoveAtom(17)
# env.StateSpace.RemoveAtom(16)
# env.StateSpace.RemoveAtom(15)
# env.StateSpace.RemoveAtom(14)
# env.StateSpace.RemoveAtom(13)
#Chem.SanitizeMol(env.StateSpace)
env.StateSpace

In [ ]:
for atom in env.StateSpace.GetAtoms():
    print(atom.GetDegree(),atom.GetSymbol(),atom.GetIsAromatic())

In [ ]:
t_mol = Chem.RWMol(Chem.MolFromSmiles('FC(CBr)c1ccccc1',sanitize = True))
t_mol

In [ ]:
env.reset()
env.addBenzine()
env.addEdge(2,0)
env.StateSpace

In [ ]:
t_mol = Chem.RWMol(Chem.MolFromSmiles('FC(CBr)c1ccccc1',sanitize = True))
env = svtr.ChemEnv
env.reset()
env.StateSpace = t_mol
# env.StateSpace
env.addEdge(2,7)
env.StateSpace

In [ ]:
env = svtr.ChemEnv
env.reset()
# env.addPyrrole()
env.addBenzine()
env.addEdge(1,2)
# env.addNode('C')
# env.addEdge(2,4)
#env.addNode('C')
#env.addEdge(1,3)
env.StateSpace

In [ ]:
mol2 = SanitizeNoKEKU(mol2)

In [ ]:
mol2

In [ ]:

mol2 = Chem.RWMol(Chem.MolFromSmiles('O=CC(=Bc1ccccc1P)P(Br)c1ccccc1.[NaH]', sanitize = True))
mol1 = Chem.RWMol(Chem.MolFromSmiles('CC.c1ccnc1', sanitize = False))
mol2.UpdatePropertyCache()
#mol2.AddAtom(Chem.Atom('C'))
#mol2.AddBond(0,5,Chem.BondType.SINGLE)
# print(mol2.NeedsUpdatePropertyCache())
# mol2.UpdatePropertyCache()
Chem.SanitizeMol(mol2)
mol1.AddBond(0,5,Chem.BondType.SINGLE)
Chem.SanitizeMol(mol1)
mol2

In [ ]:
for atom in mol2.GetAtoms():
    print(atom.GetSymbol(),atom.GetImplicitValence())

In [ ]:
SanitizeNoKEKU(mol2)
cycles = list(mol2.GetRingInfo().AtomRings())
for cycle in cycles:
    for atom_idx in cycle:
        bonds = mol2.GetAtomWithIdx(atom_idx).GetBonds()
        for bond_x in bonds:
            if bond_x.GetBondType() == Chem.BondType.DOUBLE:
                print("fraraf")

In [ ]:
for atom in mol2.GetAtoms():
    atom.UpdatePropertyCache()
    print(atom.GetExplicitValence())

In [ ]:
for bond in atom.GetBonds():
    print(bond.GetBondType())

In [ ]:
#env.reset()
env.addPyrrole()

In [ ]:
env.StateSpace

In [ ]:
env.step(17)

In [ ]:
mol = Chem.MolFromSmiles('n1cccc1', sanitize = False)
mol.UpdatePropertyCache()
for bond in mol.GetBonds():
    print(bond.GetBondType())

In [ ]:
mol

In [ ]:
Chem.MolFromSmiles('[nH]1cccc1')

In [ ]:
def SanitizeNoKEKU(mol):
    s_dict = {'SANITIZE_ADJUSTHS': Chem.rdmolops.SanitizeFlags.SANITIZE_ADJUSTHS,
    'SANITIZE_ALL': Chem.rdmolops.SanitizeFlags.SANITIZE_ALL, 
    'SANITIZE_CLEANUP': Chem.rdmolops.SanitizeFlags.SANITIZE_CLEANUP, 
    'SANITIZE_CLEANUPCHIRALITY': Chem.rdmolops.SanitizeFlags.SANITIZE_CLEANUPCHIRALITY, 
    'SANITIZE_FINDRADICALS': Chem.rdmolops.SanitizeFlags.SANITIZE_FINDRADICALS, 
    'SANITIZE_KEKULIZE': Chem.rdmolops.SanitizeFlags.SANITIZE_KEKULIZE, 
    'SANITIZE_NONE': Chem.rdmolops.SanitizeFlags.SANITIZE_NONE, 
    'SANITIZE_PROPERTIES': Chem.rdmolops.SanitizeFlags.SANITIZE_PROPERTIES, 
    'SANITIZE_SETAROMATICITY': Chem.rdmolops.SanitizeFlags.SANITIZE_SETAROMATICITY, 
    'SANITIZE_SETCONJUGATION': Chem.rdmolops.SanitizeFlags.SANITIZE_SETCONJUGATION, 
    'SANITIZE_SETHYBRIDIZATION': Chem.rdmolops.SanitizeFlags.SANITIZE_SETHYBRIDIZATION, 
    'SANITIZE_SYMMRINGS': Chem.rdmolops.SanitizeFlags.SANITIZE_SYMMRINGS}
    
    #mol = Chem.SanitizeMol(mol,s_dict['SANITIZE_KEKULIZE'])
    mol = Chem.SanitizeMol(mol, s_dict['SANITIZE_ADJUSTHS'] | s_dict['SANITIZE_SETAROMATICITY'] | 
                           s_dict['SANITIZE_CLEANUP'] | s_dict['SANITIZE_CLEANUPCHIRALITY'] | 
                           s_dict['SANITIZE_FINDRADICALS'] | s_dict['SANITIZE_NONE'] | 
                           s_dict['SANITIZE_PROPERTIES'] | s_dict['SANITIZE_SETCONJUGATION'] | 
                           s_dict['SANITIZE_SETHYBRIDIZATION'] | s_dict['SANITIZE_SYMMRINGS'] 
                          )
    return mol

In [ ]:
True | False

In [ ]:
mol = Chem.RWMol(Chem.MolFromSmiles('CC.c1ccnc1', sanitize = False))
#mol.AddBond(8,mol.GetNumAtoms()-1,Chem.BondType.SINGLE)
print(SanitizeNoKEKU(mol))
print(mol.GetAromaticAtoms().__len__())
mol

In [ ]:
from rdkit import Chem
m = Chem.MolFromSmiles('CN(C)(C)C', sanitize=False)
problems = Chem.DetectChemistryProblems(m)
print(len(problems))
m

In [ ]:
SanitizeNoKEKU(m)

In [ ]:
Chem.SanitizeFlags.SANITIZE_ADJUSTHS

In [ ]:
print(problems[0].GetType())
#print(problems[0].GetAtomIdx())
print(problems[0].Message())

In [ ]:
Chem.MolFromSmiles('CN1C=CC=CC1=O')

In [ ]:
Chem.MolFromSmiles('CN(C)(C)C', sanitize=False)

In [ ]:
# wants: a single class for pretraining and rl training
#     also want a singler logger for everything
#     also should put in cross validation for the supervised portion
#     means a logger instance in the init method

class SupervisedToReinforcement():
    def __init__(self, PPO_env, PPO_Train_Steps, policy_model,rewards, run_title):
        
        
        self.writer = SummaryWriter('./run_title')
        self.reward_module =  FinalRewardModule(sef.writer,rewards)
        
        
        self.PPO_env = PPO_env
        self.PPO_Train_Steps = PPO_Train_Steps        
        self.SV_trainer = Supervised_trainer(policy_model)
       
    
        self.SV_trainer.writer = self.writer
        self.PPO_env.env.RewardModule = self.reward_module
        self.PPO_env.actor = self.policy_model
    
    def Train():
        sv_trainer.Train(20,16, 1,24)
        self.PPO_env.learn(self.PPO_Train_Steps)
        

In [ ]:
class AdversarialTraining():
    def __init__(self, PPO_agent,Disc, epochs, G_steps,
                 D_steps, K, G_pretrain_steps, D_train_size,
                 D_batch_size,pre_train_env, smiles_values):
        
        self.PPO_agent = PPO_agent
        self.Disc = Disc
        
        self.epochs = epochs
        self.G_steps = G_steps
        self.D_steps = D_steps
        self.K = K 
        
        self.pre_train_env = pre_train_env
        
        self.D_batch_size = D_batch_size 
        self.D_train_size = D_train_size
        self.smiles_values = smiles_values
        
    def mini_batch_reward_train(self, batch_size, num_batch):
        for j in range(num_batch):
            graphs = self.PPO_agent.generate_graphs(batch_size)
            for model in self.reward_models:
                model.TrainOnBatch(graphs)
            
    def _preTrain(self):
        
        
        env,batch_size,timesteps_per_batch,clip,a_lr,c_lr,
        n_updates_per_iteration,max_timesteps_per_episode,gamma

        
        t_dict = vars(self.PPO_agent)
        PPO_agent_pre = PPO_MAIN(t_dict['env'],t_dict['batch_size'],t_dict['timesteps_per_batch'],
                                 t_dict['clip'],t_dict['a_lr'], t_dict['c_lr'],
                                 t_dict['n_updates_per_iteration'],t_dict['max_timesteps_per_episode'],
                                 t_dict['gamma'])
        
        
        
        PPO_agent_pre.learn(G_pretrain_steps)
        self.PPO_agent.assignActor(PPO_agent_pre.actor)
        
        
        
    def pull_real_samples(self, g_number):
        graphs = smiles_to_graph([self.smiles_values[random.randint(0,len(self.smiles_values))] for _ in range(g_number)])
        print(len(graphs), "graph len")
            
        return graphs
    def i_hate_python(self):
        a = self.PPO_agent.generate_graphs(10)
    def train(self, epochs):
        self._preTrain()
        for epoch in range(epochs):
            print('G_train')
            self.PPO_agent.learn(G_steps) 
            
            print('D_train')
            for d_step in range(self.D_steps):
                
                x_fake = self.PPO_agent.generate_graphs(self.D_steps)
                x_real = self.pull_real_samples(self.D_train_size)
                for k_step in range(self.K):
                    slices = list(range(0,self.D_train_size,self.D_batch_size)) + [self.D_train_size]
                    for idx in range(1,len(slices)):
                        slice_= slice(slices[idx-1],slices[idx])
                        print(slice_)
                        
                        
                        x_fake_batch = x_fake[slice_]
                        if x_fake_batch != []:
                            Y_fake_batch = torch.zeros(len(x_fake_batch),1)


                            x_real_batch = x_real[slice_]
                            Y_real_batch = torch.ones(len(x_real_batch),1)


                            self.Disc.train(x_fake_batch, Y_fake_batch)
                            self.Disc.train(x_real_batch,Y_real_batch)
                        
                
                
            
                
        